In [1]:
import numpy as np

from keras.models import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from models.unet_se import *
from models.unet import *
from models.resnet_fcn import *
from models.resnet_se_fcn import *
from models.resnet_fcn import *
from models.vgg19_fcn import *
from models.vgg19_se_fcn import *
from models.unet_resnet_se import *

from datahandler import DataHandler
from data_loader import *
from params import *
import os
import cv2
import skimage.io as io
from tqdm import tqdm

from medpy.io import save

from math import ceil, floor
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, jaccard_similarity_score

from scipy.ndimage import _ni_support
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion,\
     generate_binary_structure

from skimage.morphology import cube, binary_closing
from skimage.measure import label

import warnings
warnings.filterwarnings("ignore")

plt.gray()

Using TensorFlow backend.


In [2]:
def destiny_directory(model_name, dice_score):
    pre = './data/eval/'+model_name+'/'
    if dice_score >= 98:
        return pre + 'dice_98_100/'
    elif dice_score >= 96:
        return pre + 'dice_96_98/'
    elif dice_score >= 94:
        return pre + 'dice_94_96/'
    elif dice_score >= 92:
        return pre + 'dice_92_94/'
    elif dice_score >= 90:
        return pre + 'dice_90_92/'
    elif dice_score >= 88:
        return pre + 'dice_88_90/'
    elif dice_score >= 85:
        return pre + 'dice_85_88'
    elif dice_score >= 80:
        return pre + 'dice_80_85/'
    elif dice_score >= 70:
        return pre + 'dice_70_80/'
    elif dice_score >= 60:
        return pre + 'dice_60_70/'
    else:
        return pre + 'dice_less_60'

In [3]:
def getModel(name):
    print('Working with %s'%name)
    if name == 'unet' or name == 'unet_focal':
        model = getUnet()
    elif name == 'unet_se':
        model = getSEUnet()
    elif name == 'resnetFCN':
        model = getResnet50FCN()
    elif name == 'resnetSEFCN':
        model = getResnetSE50FCN()
    elif name == 'vgg19FCN':
        model = getVGG19FCN()
    elif name == 'vgg19SEFCN':
        model = getVGG19SEFCN()
    elif name == 'unet_resnet':
        model = getUnetRes()
    elif name == 'unet_resnet_se':
        model = getUnetRes(se_version = True)
    # elif name == 'unetResnet18':
    #     model = getUnetResnet18()
    # elif name == 'unetResnet18SE':
    #     model = getUnetResnet18(se_version = True)
    else:
        print('error')
        return -1

    return model


In [4]:
def getGenerator(images, bs=1):
    image_datagen = ImageDataGenerator(rescale=1./255)
    image_datagen.fit(images, augment = True)
    image_generator = image_datagen.flow(x = images, batch_size=bs,
            shuffle = False)

    return image_generator


In [5]:
def getDiceScore(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)


In [6]:
 def hd(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

def hd95(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity)
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity)
    hd95 = np.percentile(np.hstack((hd1, hd2)), 95)
    return hd95

def __surface_distances(result, reference, voxelspacing=None, connectivity=1):
    result = np.atleast_1d(result.astype(np.bool))
    reference = np.atleast_1d(reference.astype(np.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = np.asarray(voxelspacing, dtype=np.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()

    footprint = generate_binary_structure(result.ndim, connectivity)

    if 0 == np.count_nonzero(result):
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == np.count_nonzero(reference):
        raise RuntimeError('The second supplied array does not contain any binary object.')

    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)

    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]

    return sds

In [7]:
image_files, mask_files = load_data_files('data/kfold_data/')
print(len(image_files))
print(len(mask_files))
skf = getKFolds(image_files, mask_files, n=10)

kfold_indices = []
for train_index, val_index in skf.split(image_files, mask_files):
    kfold_indices.append({'train': train_index, 'val': val_index})

291
291


In [8]:
def predictMask(model, image):  
    image_gen = getGenerator(image)
    return model.predict_generator(image_gen, steps=len(image))

In [9]:
def prepareForSaving(image):
    image = np.squeeze(image)
    image = np.swapaxes(image, -1, 0)
    
    return image

def predictAll(model, model_name, data, num_data=0):
    dice_scores = []
    names = []
    hd_scores = []
    hd95_scores = []

    for image_file, mask_file in tqdm(data, total=num_data):
        
        fname = image_file[image_file.rindex('/')+1 : image_file.index('.')]
        image, hdr = dh.getImageData(image_file)
        gt_mask, _ = dh.getImageData(mask_file, is_mask=True)

        assert image.shape == gt_mask.shape
        
        if image.shape[1] != 256:
            continue
        
        pred_mask = predictMask(model, image)
        pred_mask[pred_mask>=0.5] = 1
        pred_mask[pred_mask<0.5] = 0
        
        
        
        #closing and defrag squeze of mask
        pred_mask = binary_closing(np.squeeze(pred_mask), cube(2))
        
        try:
            labels = label(pred_mask)
            pred_mask = (labels == np.argmax(np.bincount(labels.flat)[1:])+1).astype(int)
        
        except:
            pred_mask = pred_mask
            
        gt_mask = np.squeeze(gt_mask)
            
        dice_score = getDiceScore(gt_mask, pred_mask)
        
        if dice_score == 0:
            dice_scores.append(dice_score)
            hd_scores.append(200)
            hd95_scores.append(200) 
            continue
        
        names.append(fname)
        dice_scores.append(dice_score)
        
        hd_score = hd(gt_mask, pred_mask)
        hd_scores.append(hd_score)
        
        hd95_score = hd95(gt_mask, pred_mask)
        hd95_scores.append(hd95_score)
        
        int_dice_score = floor(dice_score * 100)
        save_path = destiny_directory(model_name, int_dice_score)
        
        pred_mask = prepareForSaving(pred_mask)
        image = prepareForSaving(image)
        gt_mask = prepareForSaving(gt_mask)
            
        save(pred_mask, os.path.join(save_path, fname + '_' + model_name + '_' 
            + str(int_dice_score) + '.nii'), hdr)
        save(image, os.path.join(save_path, fname + '_img.nii'), hdr)
        save(gt_mask, os.path.join(save_path, fname + '_mask.nii'), hdr)

    return dice_scores, hd_scores, hd95_scores, names

In [10]:
#Get data and generators

model_types = ['vgg19SEFCN', 'unet_se', 'resnetSEFCN', 'unet_resnet_se']
for model_type in model_types:
    dh = DataHandler()
    all_dice = []
    all_hd = []
    all_hd95 = []

    for i in range(len(kfold_indices)):
        exp_name = 'kfold_%s_dice_DA_K%d'%(model_type, i)

        #get parameters
        params = getParams(exp_name, unet_type=model_type)

        val_img_files = np.take(image_files, kfold_indices[i]['val'])
        val_mask_files = np.take(mask_files, kfold_indices[i]['val'])


        model = getModel(model_type)

        print('loading weights from %s'%params['checkpoint']['name'])
        model.load_weights(params['checkpoint']['name'])

        data = zip(val_img_files, val_mask_files)

        dice_score, hd_score, hd95_score, names = predictAll(model, model_type, data, num_data=len(val_mask_files))

        print('Finished K%d'%i)

        all_dice += dice_score
        all_hd += hd_score
        all_hd95 += hd95_score

    print('dice')
    for i in range(len(all_dice)):
        print(all_dice[i])
    print()

    print('hd')
    for i in range(len(all_hd)):
        print(all_hd[i])
    print()

    print('hd95')
    for i in range(len(all_hd95)):
        print(all_hd95[i])
    print()

    print('names')
    for i in range(len(names)):
        print(names[i])
    print()

    print('Final results for %s'%model_type)
    print('dice %f'%np.mean(all_dice))
    print('hd %f'%np.mean(all_hd))
    print('hd95 %f'%np.mean(all_hd95))


Working with vgg19SEFCN
loading weights from ./logs/vgg19SEFCN/kfold_vgg19SEFCN/kfold_vgg19SEFCN_dice_DA_K0/kfold_vgg19SEFCN_dice_DA_K0_weights.h5


100%|██████████| 30/30 [01:42<00:00,  3.19s/it]


Finished K0
Working with vgg19SEFCN
loading weights from ./logs/vgg19SEFCN/kfold_vgg19SEFCN/kfold_vgg19SEFCN_dice_DA_K1/kfold_vgg19SEFCN_dice_DA_K1_weights.h5


100%|██████████| 29/29 [01:45<00:00,  3.52s/it]


Finished K1
Working with vgg19SEFCN
loading weights from ./logs/vgg19SEFCN/kfold_vgg19SEFCN/kfold_vgg19SEFCN_dice_DA_K2/kfold_vgg19SEFCN_dice_DA_K2_weights.h5


...

Finished K7
Working with unet_resnet_se
loading weights from ./logs/unet_resnet_se/kfold_unet_resnet_se/kfold_unet_resnet_se_dice_DA_K8/kfold_unet_resnet_se_dice_DA_K8_weights.h5


100%|██████████| 29/29 [01:57<00:00,  4.74s/it]


Finished K8
Working with unet_resnet_se
loading weights from ./logs/unet_resnet_se/kfold_unet_resnet_se/kfold_unet_resnet_se_dice_DA_K9/kfold_unet_resnet_se_dice_DA_K9_weights.h5


100%|██████████| 29/29 [01:44<00:00,  3.23s/it]

Finished K9
dice
0.9507663923660112
0.9561730163276998
0.9134452904975487
0.8897655422780216
0.9522418189545261
0.9716761397511825
0.9701053720952112
0.9419884647952581
0.9690663867644287
0.9604395722673383
0.9633172555803675
0.9521105215354463
0.9498654393697102
0.9738250125101042
0.9685224058579467
0.9398012228297467
0.9471761394262442
0.936710060293055
0.9512560188649938
0.9662063987551206
0.41052534731806056
0.9736789745114407
0.9754372744128128
0.9690966818672997
0.9481314729472394
0.8725708160470056
0.9556933793651279
0.9480263444304169
0.9603521981777927
0.8680387147063919
0.9431568381430364
0.957729539646123
0.9350753264744343
0.9657668725114896
0.9481468874981627
0.9609192492867802
0.9511149936842602
0.9620568667676438
0.7718180853161064
0.9488018613113712
0.9541698031938233
0.9610673349558623
0.9415199334789243
0.9419754236674016
0.965104644157477
0.9313024904664452
0.9398839853928569
0.9512965109035733
0.9645479443702014
0.9490483517946429
0.9702503436497688
0.95828037205545